In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-120478")
exp = Experiment(workspace=ws, name="quick-starts-ws-120478")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-120478
Azure region: southcentralus
Subscription id: 8ca4d354-cab8-4c39-9d61-ea227d2daaad
Resource group: aml-quickstarts-120478


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
compute_cluster = ComputeTarget.create(ws, 'compute-cluster', compute_config)

In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# Specify parameter sampler
ps = RandomParameterSampling({
    "--C": uniform(0.5, 2.0),
    "--max_iter": choice(50, 100, 150, 200)
})

# Specify a Policy
policy = BanditPolicy(evaluation_interval = 3, slack_amount = 0.2)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py

est = SKLearn(source_directory='./',
              script_params={
                  "--C": 1.0,
                  "--max_iter": 100
              },
              entry_script='train.py',
              compute_target=compute_cluster)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling=ps,
                                     policy=policy,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=12,
                                     max_concurrent_runs=4,
                                     estimator=est)

In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(hyperdrive_config)
RunDetails(hyperdrive_run).show()

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.
WARNING - If 'arguments' has been provided here and arguments have been specified in 'run_config', 'arguments' provided in ScriptRunConfig initialization will take precedence.
The same input parameter(s) are specified in estimator/run_config script params and HyperDrive parameter space. HyperDrive parameter space definition will override these duplicate entries. ['--C', '--max_iter'] is the list of overridden parameter(s).


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

AmlCompute is getting created. Consider calling wait_for_completion() first



In [5]:
import joblib
# Get your best run and save the model from that run.

best_hyperdrive_run = hyperdrive_run.get_best_run_by_primary_metric()
joblib.dump(value=best_hyperdrive_run.get_details()['runDefinition']['arguments'], filename=os.path.join('training', 'best_hyperdrive_model.joblib'))

['training/best_hyperdrive_model.joblib']

In [5]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

dataset = TabularDatasetFactory.from_delimited_files(
    "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")

In [6]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(dataset)
x['y'] = y

In [9]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=x,
    label_column_name='y',
    n_cross_validations=5)

In [10]:
# Submit your automl run

automl_run = exp.submit(automl_config)
RunDetails(automl_run).show()

WARNING - Running on local machine. Note that local runs always run synchronously even if you use the parameter 'show_output=False'


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [15]:
# Retrieve and save your best automl model.
best_automl_run, fitted_model = automl_run.get_output()
joblib.dump(value=fitted_model, filename='./training/best_automl_model.joblib')

['./training/best_automl_model.joblib']

In [11]:
compute_cluster.delete()

Current provisioning state of AmlCompute is "Deleting"

